# Brazilian REH Analyzer - Jupyter Notebook Example

This notebook demonstrates how to use the Brazilian REH Analyzer for interactive analysis of inflation forecast rationality.

## Overview

The Rational Expectations Hypothesis (REH) states that economic agents form expectations optimally using all available information. This tool tests whether Brazilian Focus Bulletin inflation forecasts satisfy REH properties:

1. **Unbiasedness**: Forecast errors have zero mean
2. **Efficiency**: Forecast errors are unpredictable (no autocorrelation)
3. **Orthogonality**: Forecast errors are uncorrelated with available information


## Setup and Installation

First, make sure you have installed the Brazilian REH Analyzer:

```bash
pip install brazilian-reh-analyzer
# or for development
pip install -e .
```

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Import the Brazilian REH Analyzer
from brazilian_reh_analyzer import BrazilianREHAnalyzer
from brazilian_reh_analyzer.tests import REHTests
from brazilian_reh_analyzer.visualizations import REHVisualizations

# Set up plotting style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
%matplotlib inline

print("✅ Setup complete!")

## 1. Initialize the Analyzer

Let's start by initializing the analyzer for a recent period:

In [ ]:
# Initialize analyzer for recent period
analyzer = BrazilianREHAnalyzer(
    start_date="2020-01-01",
    end_date="2024-12-31",
    cache_dir="notebook_cache"
)

print(f"Analyzer initialized for period: {analyzer.start_date.date()} to {analyzer.end_date.date()}")

## 2. Fetch and Explore Data

Now let's fetch the data from the Brazilian Central Bank APIs. This may take a few moments:

In [ ]:
# Fetch IPCA data (realized inflation)
print("📊 Fetching IPCA 12-month accumulated data...")
ipca_data = analyzer.fetch_ipca_data()

print(f"✅ Fetched {len(ipca_data)} IPCA observations")
print(f"   Period: {ipca_data.index.min().date()} to {ipca_data.index.max().date()}")
print(f"   Mean IPCA: {ipca_data.mean():.2f}%")
print(f"   IPCA range: {ipca_data.min():.2f}% to {ipca_data.max():.2f}%")

In [ ]:
# Fetch Focus Bulletin data (forecasts)
print("📈 Fetching Focus Bulletin expectations data...")
focus_data = analyzer.fetch_focus_data()

print(f"✅ Fetched {len(focus_data)} Focus observations")
print(f"   Period: {focus_data.index.min().date()} to {focus_data.index.max().date()}")
print(f"   Mean forecast: {focus_data['Mediana'].mean():.2f}%")
print(f"   Average respondents: {focus_data['numeroRespondentes'].mean():.1f}")

# Display first few rows
print("\n📋 Sample Focus data:")
display(focus_data.head())

## 3. Align Forecasts with Realizations

The key challenge is properly aligning 12-month ahead forecasts with their corresponding realizations:

In [ ]:
# Align forecast and realization data
print("🔄 Aligning forecast and realization data...")
aligned_data = analyzer.align_forecast_realization_data()

print(f"✅ Successfully aligned {len(aligned_data)} forecast-realization pairs")
print(f"   Analysis period: {aligned_data.index.min().date()} to {aligned_data.index.max().date()}")

# Display sample aligned data
print("\n📋 Sample aligned data:")
display(aligned_data.head(10))

# Basic statistics
print("\n📊 Basic Statistics:")
print(f"   Mean forecast error: {aligned_data['forecast_error'].mean():.3f} p.p.")
print(f"   Std forecast error: {aligned_data['forecast_error'].std():.3f} p.p.")
print(f"   Correlation (forecast, realized): {aligned_data['forecast'].corr(aligned_data['realized']):.3f}")

## 4. Initial Data Visualization

Let's create some initial plots to understand the data:

In [ ]:
# Create initial visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Forecast vs Realized scatter
axes[0, 0].scatter(aligned_data['forecast'], aligned_data['realized'], alpha=0.6, s=50)
min_val = min(aligned_data['forecast'].min(), aligned_data['realized'].min())
max_val = max(aligned_data['forecast'].max(), aligned_data['realized'].max())
axes[0, 0].plot([min_val, max_val], [min_val, max_val], 'r--', alpha=0.8, label='Perfect Forecast')
axes[0, 0].set_xlabel('Focus Forecast (%)')
axes[0, 0].set_ylabel('Realized IPCA (%)')
axes[0, 0].set_title('Forecast vs Realization')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Time series
axes[0, 1].plot(aligned_data.index, aligned_data['forecast'], label='Forecast', alpha=0.8, linewidth=2)
axes[0, 1].plot(aligned_data.index, aligned_data['realized'], label='Realized', alpha=0.8, linewidth=2)
axes[0, 1].set_ylabel('IPCA 12-month (%)')
axes[0, 1].set_title('Forecasts and Realizations Over Time')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].tick_params(axis='x', rotation=45)

# Forecast errors
errors = aligned_data['forecast_error']
axes[1, 0].plot(errors.index, errors.values, color='red', alpha=0.7, linewidth=1.5)
axes[1, 0].axhline(y=0, color='black', linestyle='--', alpha=0.7)
axes[1, 0].axhline(y=errors.mean(), color='green', linestyle=':', alpha=0.8, 
                   label=f'Mean: {errors.mean():.3f} p.p.')
axes[1, 0].set_ylabel('Forecast Error (p.p.)')
axes[1, 0].set_title('Forecast Errors Over Time')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].tick_params(axis='x', rotation=45)

# Error distribution
axes[1, 1].hist(errors, bins=20, alpha=0.7, density=True, color='lightcoral', edgecolor='black')
axes[1, 1].axvline(x=0, color='red', linestyle='--', alpha=0.8, label='Zero Error')
axes[1, 1].axvline(x=errors.mean(), color='green', linestyle=':', alpha=0.8, 
                   label=f'Mean: {errors.mean():.3f}')
axes[1, 1].set_xlabel('Forecast Error (p.p.)')
axes[1, 1].set_ylabel('Density')
axes[1, 1].set_title('Distribution of Forecast Errors')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Run Individual Econometric Tests

Now let's run the individual REH tests to understand forecast rationality:

In [ ]:
# 5.1 Mincer-Zarnowitz Test
print("1️⃣ MINCER-ZARNOWITZ TEST")
print("=" * 40)
print("Tests: H₀: (α, β) = (0, 1) in regression: Realized = α + β × Forecast")
print()

mz_results = REHTests.mincer_zarnowitz_test(
    aligned_data['forecast'], 
    aligned_data['realized']
)

if 'error' not in mz_results:
    print(f"📊 Regression Results:")
    print(f"   α (intercept): {mz_results['alpha']:.4f} (SE: {mz_results['alpha_stderr']:.4f})")
    print(f"   β (slope): {mz_results['beta']:.4f} (SE: {mz_results['beta_stderr']:.4f})")
    print(f"   R-squared: {mz_results['r_squared']:.4f}")
    print(f"")
    print(f"🧪 Test Results:")
    print(f"   α = 0 test p-value: {mz_results['alpha_pvalue']:.4f}")
    print(f"   β = 1 test p-value: {mz_results['beta_pvalue']:.4f}")
    print(f"   Joint test p-value: {mz_results['joint_test_pvalue']:.4f}")
    print(f"")
    print(f"🎯 Result: {'✅ RATIONAL' if mz_results['passes_joint_test'] else '❌ NOT RATIONAL'}")
    
    if mz_results['joint_test_pvalue'] < 0.05:
        print(f"   📝 Forecasts violate unbiasedness/efficiency (reject H₀)")
    else:
        print(f"   📝 Forecasts are unbiased and efficient")
else:
    print(f"❌ Test failed: {mz_results['error']}")

print("\n" + "="*60 + "\n")

In [ ]:
# 5.2 Autocorrelation Test
print("2️⃣ AUTOCORRELATION TEST (Ljung-Box)")
print("=" * 50)
print("Tests: H₀: No autocorrelation in forecast errors")
print()

autocorr_results = REHTests.autocorrelation_test(
    aligned_data['forecast_error'], 
    max_lags=10
)

if 'error' not in autocorr_results:
    print(f"🧪 Test Results:")
    print(f"   Ljung-Box statistic: {autocorr_results['ljung_box_stat']:.4f}")
    print(f"   p-value: {autocorr_results['ljung_box_pvalue']:.4f}")
    print(f"   Lags tested: {autocorr_results['max_lags_tested']}")
    print(f"   Observations: {autocorr_results['n_observations']}")
    print(f"")
    print(f"🎯 Result: {'✅ EFFICIENT' if autocorr_results['passes_efficiency_test'] else '❌ NOT EFFICIENT'}")
    
    if autocorr_results['significant_autocorr']:
        print(f"   📝 Significant autocorrelation detected in forecast errors")
    else:
        print(f"   📝 No significant autocorrelation in forecast errors")
else:
    print(f"❌ Test failed: {autocorr_results['error']}")

print("\n" + "="*60 + "\n")

In [ ]:
# 5.3 Bias Test
print("3️⃣ BIAS TEST (Holden-Peel)")
print("=" * 40)
print("Tests: H₀: Mean forecast error = 0")
print()

bias_results = REHTests.bias_test(aligned_data['forecast_error'])

if 'error' not in bias_results:
    print(f"🧪 Test Results:")
    print(f"   Mean forecast error: {bias_results['mean_error']:.4f} p.p.")
    print(f"   Standard error: {bias_results['std_error']:.4f} p.p.")
    print(f"   t-statistic: {bias_results['t_statistic']:.4f}")
    print(f"   p-value: {bias_results['p_value']:.4f}")
    print(f"   95% Confidence Interval: [{bias_results['confidence_interval_95'][0]:.4f}, {bias_results['confidence_interval_95'][1]:.4f}]")
    print(f"")
    print(f"🎯 Result: {'✅ UNBIASED' if bias_results['passes_unbiasedness_test'] else '❌ BIASED'}")
    
    if bias_results['is_biased']:
        print(f"   📝 Systematic {bias_results['bias_direction']} detected")
    else:
        print(f"   📝 No systematic bias in forecasts")
else:
    print(f"❌ Test failed: {bias_results['error']}")

print("\n" + "="*60 + "\n")

## 6. Comprehensive Analysis

Now let's run the complete analysis suite and get the overall rationality assessment:

In [ ]:
# Run comprehensive analysis
print("🔄 Running comprehensive REH analysis...")

results = analyzer.comprehensive_analysis(fetch_data=False)  # Data already fetched

print("✅ Analysis completed!")
print("\n" + "="*60)
print("📋 COMPREHENSIVE RESULTS SUMMARY")
print("="*60)

# Display descriptive statistics
desc_stats = results['descriptive_stats']
print(f"\n📊 Descriptive Statistics:")
print(f"   Analysis period: {desc_stats['date_range']}")
print(f"   Total observations: {desc_stats['n_observations']}")
print(f"   Mean forecast error: {desc_stats['error_mean']:.4f} p.p.")
print(f"   Error std deviation: {desc_stats['error_std']:.4f} p.p.")
print(f"   Error range: {desc_stats['error_min']:.2f} to {desc_stats['error_max']:.2f} p.p.")
print(f"   Average respondents: {desc_stats['mean_respondents']:.1f}")

# Display rationality assessment
rationality = results['rationality_assessment']
print(f"\n🎯 Rationality Assessment:")
print(f"   Unbiased: {'✅ PASS' if rationality['unbiased'] else '❌ FAIL'}")
print(f"   MZ Test: {'✅ PASS' if rationality['mz_rational'] else '❌ FAIL'}")
print(f"   Efficient: {'✅ PASS' if rationality['efficient'] else '❌ FAIL'}")
print(f"   ")
print(f"   🏆 OVERALL: {'✅ RATIONAL' if rationality['overall_rational'] else '❌ NOT RATIONAL'}")

# Summary interpretation
print(f"\n📝 Summary Interpretation:")
if rationality['overall_rational']:
    print(f"   Focus Bulletin forecasts satisfy the Rational Expectations Hypothesis.")
    print(f"   Forecasts are unbiased, efficient, and pass all standard tests.")
else:
    failed_tests = []
    if not rationality['unbiased']:
        failed_tests.append("systematic bias")
    if not rationality['mz_rational']:
        failed_tests.append("Mincer-Zarnowitz inefficiency")
    if not rationality['efficient']:
        failed_tests.append("autocorrelation")
    
    print(f"   Focus Bulletin forecasts violate the Rational Expectations Hypothesis.")
    print(f"   Issues detected: {', '.join(failed_tests)}")
    print(f"   This suggests forecasters are not using information optimally.")

## 7. Advanced Visualizations

Let's create the comprehensive diagnostic plots:

In [ ]:
# Generate comprehensive diagnostic plots
print("📈 Generating comprehensive diagnostic plots...")

fig = analyzer.plot_enhanced_diagnostics(show_plots=True)

print("✅ Diagnostic plots generated!")

## 8. Export Results

Finally, let's export our results for further use:

In [ ]:
# Export results summary
analyzer.export_results_summary("notebook_analysis_results.txt")
print("💾 Results summary saved to: notebook_analysis_results.txt")

# Export plots
analyzer.export_plots("notebook_plots/", dpi=300)
print("🖼️ Diagnostic plots saved to: notebook_plots/")

# Save aligned data for further analysis
analyzer.save_data("notebook_aligned_data.csv")
print("📋 Aligned data saved to: notebook_aligned_data.csv")

print("\n✅ All results exported successfully!")

## 9. Explore Results Further

You can now explore the results in more detail:

In [ ]:
# Access detailed results
print("🔍 Available result categories:")
for key in results.keys():
    print(f"   • {key}")

print("\n📊 You can explore:")
print("   • results['mincer_zarnowitz'] - MZ test details")
print("   • results['autocorrelation'] - Ljung-Box test details")
print("   • results['bias_test'] - Bias test details")
print("   • analyzer.aligned_data - Raw aligned dataset")
print("   • analyzer.forecast_errors - Forecast error series")

# Example: Show detailed MZ results
if 'mincer_zarnowitz' in results and 'error' not in results['mincer_zarnowitz']:
    mz = results['mincer_zarnowitz']
    print("\n🔬 Detailed Mincer-Zarnowitz Results:")
    for key, value in mz.items():
        if isinstance(value, float):
            print(f"   {key}: {value:.6f}")
        else:
            print(f"   {key}: {value}")

## 10. Next Steps

This notebook demonstrated the basic usage of the Brazilian REH Analyzer. For more advanced usage, you can:

1. **Analyze different time periods** by changing the `start_date` and `end_date`
2. **Add external variables** for orthogonality testing
3. **Compare multiple periods** to study structural breaks
4. **Create custom visualizations** using the aligned data
5. **Export results** in different formats for publication

See the other examples in the `docs/examples/` directory for more advanced usage patterns.

### Key Takeaways

- The Brazilian REH Analyzer provides a comprehensive framework for testing forecast rationality
- All standard econometric tests are implemented with proper statistical inference
- Data is automatically fetched and cached from Brazilian Central Bank APIs
- Results are exportable in multiple formats suitable for academic research

Happy analyzing! 🇧🇷📊